In [ ]:
!pip install alpaca-trade-api pandas numpy schedule
!pip install praw pandas transformers torch numpy google-play-scraper
!pip install google-play-scraper

In [ ]:
from alpaca_trade_api.rest import REST
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import os
import matplotlib.pyplot as plt
import praw
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from google_play_scraper import reviews

Sentiment Algorithm

In [ ]:
#Scrape Reddit 1
def scrape_reddit(client_id, client_secret, user_agent, query="Chipotle", limit=1000):
    reddit = praw.Reddit(
        client_id=client_id,
        client_secret=client_secret,
        user_agent=user_agent
    )
    posts = []
    for submission in reddit.subreddit("all").search(query, limit=limit):
        if not submission.stickied:
            posts.append(submission.title + " " + submission.selftext)
    return pd.DataFrame({'text': posts})

#Scrape Google Play reviews 2
def scrape_google_play(app_id="com.chipotle.ordering", limit=200):
    review_data, _ = reviews(
        app_id,
        lang='en',
        country='us',
        count=limit
    )
    return pd.DataFrame({'text': [r['content'] for r in review_data]})

#Load sentiment model HAVE TO CHANGE THIS
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

label_map = {
    "LABEL_0": "negative",
    "LABEL_1": "neutral",
    "LABEL_2": "positive"
}

sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    top_k=None
)

#Sentiment analysis
def analyze_sentiment(text):
    try:
        result = sentiment_pipeline(text)[0]
        scores = {label_map[entry['label']]: entry['score'] for entry in result}
        return max(scores, key=scores.get)
    except Exception as e:
        print(f"Error: {e}")
        return "neutral"


client_id = "M8D3he3NjOj8Y4Z3PF3gzw"
client_secret = "77fOIkC8gN_SC5g_-2TDXKKZZFENXg"
user_agent = "chipotle-analysis by u/No-Chemistry-203"

#Scrape data
print("Scraping Reddit...")
reddit_df = scrape_reddit(client_id, client_secret, user_agent, query="Chipotle", limit=1000)

print("Scraping Google Play...")
google_df = scrape_google_play(app_id="com.chipotle.ordering", limit=200)

#Combine and analyze
combined_df = pd.concat([reddit_df, google_df], ignore_index=True)
combined_df['sentiment'] = combined_df['text'].apply(analyze_sentiment)

#Export
#output_path = r"C:\Users\dpods\Downloads\chipotle_2sentiment.csv"
#combined_df.to_csv(output_path, index=False, encoding='utf-8')
#print(f"Analysis complete, File saved to: {output_path}")

#Check
print("Reddit rows:", len(reddit_df))
print("Google Play rows:", len(google_df))
print("Total combined rows:", len(combined_df))

summary = combined_df['sentiment'].value_counts()
print(summary)

sentiment_counts = combined_df['sentiment'].value_counts()

Uptrend Algorithm

In [ ]:
# variables
API_KEY = "PK0CBK5ROYJZAAEBUWKB"
SECRET_KEY = "Tip93sYY2IAz6Mxgx99A3qBzvlXlVrU3grqhhd9P"
BASE_URL = "https://paper-api.alpaca.markets"
STOCK_SYMBOL = "CMG"
TRADE_AMOUNT = 100

In [ ]:
# connect to API
api = REST(API_KEY, SECRET_KEY, base_url=BASE_URL)

In [ ]:
# fetch 50-day MA function
def get_price_data(symbol):
    end_date = datetime.utcnow() - timedelta(days=1)
    start_date = end_date - timedelta(days=90)

    bars = api.get_bars(
        symbol,
        timeframe="1Day",
        start=start_date.strftime("%Y-%m-%dT%H:%M:%SZ"),
        end=end_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    ).df

    df = bars.copy()
    df['MA_50'] = df['close'].rolling(window=50).mean()

    return df


In [ ]:
# this function checks if the most recent closing price is greater than the 50-day MA
def is_uptrend(df):
    latest_close = df['close'].iloc[-1]
    latest_ma = df['MA_50'].iloc[-1]
    return latest_close > latest_ma, latest_close, latest_ma

In [ ]:
# place buy function
def place_buy(symbol, current_price):
    qty = round(TRADE_AMOUNT / current_price, 4)
    order = api.submit_order(
        symbol=symbol,
        qty=qty,
        side="buy",
        type="market",
        time_in_force="day"
    )
    return qty


In [ ]:
# sell function

def place_sell(symbol, qty):
    try:
        order = api.submit_order(
            symbol=symbol,
            qty=qty,
            side='sell',
            type='market',
            time_in_force='day'
        )
        print(f"Sell order placed for {qty} shares of {symbol}")
        return order
    except Exception as e:
        print(f"Error placing sell order for {symbol}: {e}")
        return None



In [ ]:
# csv log
def log_trade(symbol, qty, price, ma_50):
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "symbol": symbol,
        "price": price,
        "ma_50": ma_50,
        "qty": qty,
        "trade_value": round(qty * price, 2)
    }

    log_file = "trade_log.csv"
    if os.path.exists(log_file):
        df = pd.read_csv(log_file)
        df = pd.concat([df, pd.DataFrame([log_entry])], ignore_index=True)
    else:
        df = pd.DataFrame([log_entry])

    df.to_csv(log_file, index=False)
    print(f"Trade logged for {symbol}")


In [ ]:
account = api.get_account()

print("API Connected")
print(f"Account status: {account.status}")
print(f"Equity: ${account.equity}")
print(f"Buying power: ${account.buying_power}")

**Notes:**
- make list of stocks
- have run_bot loop through until it finds one with uptrend.
- If it does, buy and move on to sell loop

In [ ]:
# main
positions = {} # check positions

def run_bot(symbol):
    df = get_price_data(symbol)

    if df.empty or df['MA_50'].isna().all():
        print(f"No valid data for {symbol}. Possibly rate-limited or stock symbol is invalid.")
        return

    uptrend, price, ma = is_uptrend(df)

    if symbol in positions:
      entry_price = positions[symbol]['entry_price']
      qty = positions[symbol]['qty']
      change_pct = (price - entry_price) / entry_price

        # if change_pct >= 0.02 # this is our take profit exit
          # print(f"SELL {symbol} + 2%. Entry {en}")

    if uptrend:
        print(f"{symbol} is in uptrend. Price: {price:.2f}, MA50: {ma:.2f}")
        qty = place_trade(symbol, price)
        log_trade(symbol, qty, price, ma)
    else:
        print(f"{symbol} is NOT in uptrend. Price: {price:.2f}, MA50: {ma:.2f}")


In [ ]:
run_bot(STOCK_SYMBOL)